# K-Nearest Neighbors From Scratch Using Only NumPy

In [1]:
import numpy as np

In [2]:
data = np.genfromtxt('/Users/tcbon/Downloads/breast_cancer_coimbra.csv', delimiter=',', skip_header=1)

In [3]:
def train_test_split(data):
    # Train set split
    train_indices = np.random.choice(len(data), size=int(len(data)*.8), replace=False)
    train = data[train_indices,:]
    x_train = [x[:-1] for x in train]
    y_train = [x[-1] for x in train]
    
    # Test set split
    test_indices = set(range(len(data))) - set(train_indices)
    test = data[list(test_indices),:]
    x_test = [x[:-1] for x in test]
    y_test = [x[-1] for x in test]
    
    return x_train, x_test, y_train, y_test

In [4]:
def column_means_and_stdevs(x_train):
    # Finding mean and stdev of each column
    return np.mean(x_train, axis=0), np.std(x_train, axis=0)    

In [5]:
def apply_standardization(data):    
    # Applying standardization to data
    for column_ix in range(len(data[0])): 
        col_values = []
        for row_ix in range(len(data)):
            # Replacing old value with z-score
            data[row_ix][column_ix] = (data[row_ix][column_ix] - means[column_ix]) / stdevs[column_ix]
    return data

In [6]:
def knn_classifier(x_train, x_test, y_train, k, p=2):
    
    # Loop through each test pt
    predicted_test_classes = []
    for test_pt in x_test:
        distances = []
        # Loop through each train pt
        for train_pt in x_train: 
            column_differences = [] 
            # Loop through columns
            for i in range(len(test_pt)): 
                # Find the difference between the test and train for a particular column, 
                # raise to the power of p and append to the column_differences list.
                column_differences.append(np.abs(test_pt[i] - train_pt[i])**p) 

            # Append output of Minkowski Formula to the distances list       
            distances.append(sum(column_differences)**(1/p)) 

        # Find k-nearest neighbors indices
        nearest_neighbors_indices = np.asarray(distances).argsort()[:k] 

        # Count number of nearest neighbors for each class
        nearest_neighbors_classes = {}
        for pt in np.take(y_train, nearest_neighbors_indices):
            if pt not in nearest_neighbors_classes:
                nearest_neighbors_classes[pt] = 1
            else:
                nearest_neighbors_classes[pt] += 1

        # Find class with greatest number of nearest neighbors
        
        # If k is even and there are equal numbers of each class
        if k % 2 == 0 and len(set(nearest_neighbors_classes.values())) == 1:
            test_pt_class = np.random.randint(1,3)
        # If k is odd or even without a tie
        else:
            test_pt_class = max(nearest_neighbors_classes, key=nearest_neighbors_classes.get)
                
        predicted_test_classes.append(test_pt_class)
    
    return predicted_test_classes

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data)

In [8]:
means, stdevs = column_means_and_stdevs(x_train)

In [9]:
x_scaled_train = apply_standardization(x_train)
x_scaled_test = apply_standardization(x_test)

In [10]:
y_test_predicted = knn_classifier(x_scaled_train, x_scaled_test, y_train,3)

In [11]:
# Actual vs Predicted Values
print('actual    predicted')
for x,y in zip(y_test,y_test_predicted):
    print(x, '     ' , y)

actual    predicted
1.0       1.0
1.0       1.0
1.0       1.0
1.0       1.0
1.0       1.0
1.0       2.0
1.0       2.0
1.0       1.0
1.0       1.0
1.0       2.0
1.0       1.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       1.0
2.0       1.0
2.0       1.0


## Verifying Validity of Code by Comparing to scikit-learn

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [13]:
scalar = StandardScaler()

scalar.fit(x_train)

x_scaled_train2 = scalar.transform(x_train)
x_scaled_test2 = scalar.transform(x_test)

In [14]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_scaled_train2, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [15]:
# Outputs are the same!!!
print('sklearn   mine')
for x,y in zip(knn.predict(x_scaled_test2),y_test_predicted):
    print(x, '     ' , y)

sklearn   mine
1.0       1.0
1.0       1.0
1.0       1.0
1.0       1.0
1.0       1.0
2.0       2.0
2.0       2.0
1.0       1.0
1.0       1.0
2.0       2.0
1.0       1.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
2.0       2.0
1.0       1.0
1.0       1.0
1.0       1.0
